<a href="https://colab.research.google.com/github/Pietro-Manning/AMD/blob/main/main_amd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from PIL import Image

In [2]:
import os
import pandas as pd


os.environ['KAGGLE_USERNAME'] = "xxxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxxxxx"

DATASET_LINK = "maparla/prado-museum-pictures"


!kaggle datasets download -d {DATASET_LINK}


Dataset URL: https://www.kaggle.com/datasets/maparla/prado-museum-pictures
License(s): MIT
100% 24.9G/24.9G [05:15<00:00, 135MB/s]
100% 24.9G/24.9G [05:15<00:00, 84.9MB/s]


In [3]:
!unzip -o prado-museum-pictures.zip

!ls
import pandas as pd
try:
    df = pd.read_csv('prado.csv')
    print("DataFrame loaded successfully")
except FileNotFoundError:
    print("The file 'prado.csv' was not found. Please check the extracted files.")

Output streaming troncato alle ultime 5000 righe.
  inflating: images/images/a01de9ba-e790-457d-8b08-0312bd49f8ee.jpg  
  inflating: images/images/a0215d66-768d-4b08-9f45-5debe380e3cc.jpg  
  inflating: images/images/a0234db3-6160-47ec-a62f-ae4585b30cfe.jpg  
  inflating: images/images/a02862ca-52ef-4aeb-b1c2-0654954c8426.jpg  
  inflating: images/images/a02a876e-f3fe-40a2-9ea9-8b25a5bda827.jpg  
  inflating: images/images/a0324706-3b2b-49ca-8877-e7e031e5a487.jpg  
  inflating: images/images/a034f244-a906-4d5b-9a13-299648fd4498.jpg  
  inflating: images/images/a034f7c7-f71c-4a52-8f0d-5cd79d2f3c33.jpg  
  inflating: images/images/a036ba7c-f04c-4c4c-892a-36137ba332b7.jpg  
  inflating: images/images/a0416ffb-9003-4580-bcbd-e475c5a5c9fa.jpg  
  inflating: images/images/a04b1ee5-2e98-4d4c-a6a1-82b56b104588.jpg  
  inflating: images/images/a05097fc-491a-4e28-ad2a-8a9659e91e97.jpg  
  inflating: images/images/a052ca98-90f2-402f-8313-1cfdc050025d.jpg  
  inflating: images/images/a060b62e-9653

In [4]:
!pip install pyspark

# Step 2: Inizializza la sessione Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Prado Museum PageRank") \
    .getOrCreate()

# Step 3: Carica il dataset
df = spark.read.csv('prado.csv', header=True, inferSchema=True)

# Step 4: Mostra i primi 5 record
df.show(5)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=aeba5f086bb4e6d7e571a6a3b7eea0f8a24ea23b2b9b730e502ff6291946c90b
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+------------+--------------------+-----------------+----------------------------------+---------------------+----------------------+---------------------+-----------------------+-----------------------+-------------------------+---------------------+---------------------------+------------+---------+-----------+---------+-----------------------+--------------------------------+-----------------------+-------------------

In [7]:
from pyspark.sql.functions import col, explode, collect_list, split

# Split the work_tags column into an array of tags
df_tags = df.withColumn("tags_array", split(col("work_tags"), ", "))

# Explode the tags_array column
df_tags_exploded = df_tags.withColumn("tag", explode(col("tags_array")))

# Group by tag and collect all images associated with that tag
df_links = df_tags_exploded.groupBy("tag").agg(collect_list("work_url").alias("pictures"))

# Create edges (connections) between images that share at least one tag
edges = df_links.select(explode("pictures").alias("src"), explode("pictures").alias("dst")).distinct()

# Filter out self-loops (connections where an image links to itself)
edges = edges.filter(col("src") != col("dst"))

# Convert to RDD to use Spark's RDD APIs
edges_rdd = edges.rdd.map(lambda row: (row['src'], row['dst']))

In [8]:
# Number of iterations for PageRank calculation
num_iterations = 10
alpha = 0.15  # Damping factor

# Create an RDD of (pages, outgoing links)
links = edges_rdd.distinct().groupByKey().cache()

# Initialize each node with a rank of 1.0
ranks = links.mapValues(lambda _: 1.0)

for iteration in range(num_iterations):
    # Distribute the rank of each page across its outgoing links
    contribs = links.join(ranks).flatMap(
        lambda url_neighbors_rank: [(dest, url_neighbors_rank[1][1] / len(url_neighbors_rank[1][0])) for dest in url_neighbors_rank[1][0]]
    )

    # Recalculate the ranks using the damping factor
    ranks = contribs.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: alpha + (1 - alpha) * rank)

# Sort the results by rank
sorted_ranks = ranks.sortBy(lambda x: -x[1])

# Display the top 10 results
print(sorted_ranks.take(10))

[('Fuente: Museo Nacional del Prado"', 8.031017495255742), ('Mena Marqués', 5.8232578238429005), ('Posada Kubissa', 4.847559313804331), ('Pérez Sánchez', 4.335954024759142), ('Museo Nacional del Prado', 4.06153917548966), ('Ruiz Gómez', 4.058671945290737), ('Luna', 4.014261671170429), ('Maurer', 3.440580580961301), ('Azcue Brea', 3.09918514295106), ('Albarrán', 3.0850679578532616)]


In [9]:
sorted_ranks.toDF(["picture_id", "pagerank"]).write.csv('/content/prado_pagerank_distributed.csv')

sorted_ranks.toDF(["picture_id", "pagerank"]).show(10)

+--------------------+------------------+
|          picture_id|          pagerank|
+--------------------+------------------+
|Fuente: Museo Nac...| 8.031017495255742|
|        Mena Marqués|5.8232578238429005|
|      Posada Kubissa| 4.847559313804331|
|       Pérez Sánchez| 4.335954024759142|
|Museo Nacional de...|  4.06153917548966|
|          Ruiz Gómez| 4.058671945290737|
|                Luna| 4.014261671170429|
|              Maurer| 3.440580580961301|
|          Azcue Brea|  3.09918514295106|
|            Albarrán|3.0850679578532616|
+--------------------+------------------+
only showing top 10 rows



In [10]:
total_rank = sorted_ranks.map(lambda x: x[1]).sum()
print(f"Total PageRank: {total_rank}")

Total PageRank: 2087.0


In [11]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("PageRank") \
    .getOrCreate()

# Assume that edges_rdd is already defined

# Number of iterations and damping factor
num_iterations = 10
alpha = 0.15

# Create an RDD of (pages, outgoing links)
links = edges_rdd.distinct().groupByKey().cache()

# Initialize each node with a rank of 1.0
ranks = links.mapValues(lambda _: 1.0)

for iteration in range(num_iterations):
    # Distribute the rank of each page across its outgoing links
    contribs = links.join(ranks).flatMap(
        lambda url_neighbors_rank: [(dest, url_neighbors_rank[1][1] / len(url_neighbors_rank[1][0])) for dest in url_neighbors_rank[1][0]]
    )

    # Recalculate the ranks using the damping factor
    ranks = contribs.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: alpha + (1 - alpha) * rank)

# Find the total sum of ranks
total_rank = ranks.map(lambda x: x[1]).sum()
print(f"Total PageRank (before normalization): {total_rank}")

# Normalize the ranks
normalized_ranks = ranks.mapValues(lambda rank: rank / total_rank)

# Find the total sum of normalized ranks
total_normalized_rank = normalized_ranks.map(lambda x: x[1]).sum()
print(f"Total PageRank (normalized): {total_normalized_rank}")

# Sort the results by rank
sorted_ranks = normalized_ranks.sortBy(lambda x: -x[1])

# Display the top 10 results
sorted_ranks_df = sorted_ranks.toDF(["picture_id", "pagerank"])
sorted_ranks_df.show(10)

# Save the results to a CSV file
sorted_ranks_df.write.mode("overwrite").csv('/content/prado_pagerank_normalized.csv')

Total PageRank (before normalization): 2086.999999999998
Total PageRank (normalized): 1.0000000000000115
+--------------------+--------------------+
|          picture_id|            pagerank|
+--------------------+--------------------+
|Fuente: Museo Nac...|0.003848115714066...|
|        Mena Marqués|0.002790252910322...|
|      Posada Kubissa|0.002322740447438589|
|       Pérez Sánchez|0.002077601353502...|
|Museo Nacional de...|0.001946113644221...|
|          Ruiz Gómez|0.001944739791706...|
|                Luna|0.001923460312012...|
|              Maurer|0.001648577183019312|
|          Azcue Brea|0.001484995276929...|
|            Albarrán|0.001478230933326...|
+--------------------+--------------------+
only showing top 10 rows

